In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My\ Drive/CSCI\ 470\ Project/Epic

from IPython import display
!pip install transformers
from transformers import GPT2Model, GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
import torch
display.clear_output()

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("\nRunning on device: ", str(device).upper())

if gpu_info.find('failed') >= 0 and ram_gb < 30:
  print('\nSelect the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
  print('\nTo enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  try:
      if gpu_info.find('failed') < 0:
         print(gpu_info)
  except:
    display.clear_output()
  finally:
    print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
    print('You are using a high-RAM runtime!')


def get_bad_word_ids(tokenizer, bad_words=None, blacklist_path="./Data/Cleaned_Black_List.txt"):
    if not bad_words:
        bad_words = []
        with open(blacklist_path, "r") as reader:
            lines = reader.readlines()
            for word in lines:
                if (len(word) > 0):
                    bad_words.append(word)
    bad_word_ids = [tokenizer.encode(bad_word) for bad_word in bad_words]
    return bad_word_ids

def classify_input_tokens(input_text, threshold=0.9):
    tokens=""
    #classify the lines according to literary tokens
    literary_generator = zeroshot_generator(input_text, literary_tokens, multi_class=True)
    for i, score in enumerate(literary_generator['scores']):
        if score > threshold:
            tokens += "["+ literary_generator['labels'][i].upper() + "]"
        else: break
    return tokens

def generate_outputs(input_context, model, tokenizer, bad_words_ids, num_sequences=3, top_k=50, top_p=0.95, max_length=250):
    input_ids = tokenizer.encode(input_context, return_tensors='pt').to(device)  # encode input context
    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=max_length, 
        top_k=top_k, 
        top_p=top_p,
        temperature=0.7,
        no_repeat_ngram_size=3,
        num_return_sequences=num_sequences, 
        bad_words_ids=bad_words_ids,
        early_stopping=True
    )
    decoded_outputs = []
    decoded_outputs_tokens = []
    #tokenizer.decode(input_ids[:end])+
    for i, sample_output in enumerate(sample_outputs):
        decoded_outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True))
        decoded_outputs_tokens.append(tokenizer.decode(sample_output))

    return decoded_outputs, decoded_outputs_tokens


Running on device:  CUDA
Thu Dec  3 02:14:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                              

In [ ]:
%cd /content/drive/MyDrive/CSCI 470 Project/Epic
literary_tokens = ["characterization", "character", "setting", 
                   "exposition", "climax", "resolution", "plot", 
                   "context", "action", "weapon", 
                   "danger", "death", "suspense", 
                   "emotion", "surprise", "problem", "conflict", 
                   "perspective", "transition", "relief", "metaphor", 
                   "flashback"]

subgenre_tokens = ['Vampire', 'Ghost', 'Horror', 'Comedic Horror', 'Murder', 
                   'Werewolf', 'Apocalypse','Haunted House', 'Witch', 'Hell', 
                   'Alien', 'Gore', 'Monster']


author_list = ['Brian Evenson', 'M R James', 'Ambrose Bierce', 'Elliott O Donnell', 'Joseph Sheridan Le Fanu',
               'Edgar Allan Poe', 'Bram Stoker', 'Algernon Blackwood', 
               'Madeline Gobbo', 'Nnedi Okorafor', 'Sofia Samatar', 'Franz Kafka', 
               'Laird Barron', 'Nathan Ballingrud', 'Nellie Bly', 'William Hope Hodsgon',
               'Kelly Link', 'Arthur Machen', 'George Sylvester Viereck', 'Robert Chambers', 
               'John Meade Falkner', 'Ann Radcliffe', 'Howard Lovecraft', 'Robert Louis Stevenson',
               'Edith Birkhead', 'Jeff Vandermeer', 'Henry James', 'John William Polidori', 'W Bob Holland',
               'Prest and Rymer', 'Oliver Onions', 'Stephen King', 'Clive Barker']

literary_tokens = {"characterization": '[CHARACTERIZATION]', "character": '[CHARACTER]', "setting": '[SETTING]', 
                   "exposition": '[EXPOSTION]', "climax": '[CLIMAX]', "resolution": '[RESOLUTION]', "plot": '[PLOT]', 
                   "context": '[CONTEXT]', "action": '[ACTION]', "weapon": '[WEAPON]', "danger": '[DANGER]', "death": '[DEATH]', "suspense": '[SUSPENSE]', 
                   "emotion": '[EMOTION]', "surprise": '[SURPRISE]', "problem":'[PROBLEM]', "conflict": '[CONFLICT]', 
                   "perspective": '[PERSPECTIVE]', "transition": '[TRANSITION}', "relief": '[RELIEF]', "metaphor": '[METAPHOR]', 
                   "flashback": '[FLASHBACK]'}

subgenre_tokens = {'Vampire': '[VAMPIRE]', 'Ghost' : '[GHOST]', 'Horror' : '[HORROR]', 'Comedic Horror' : '[COMEDIC HORROR]', 'Murder' : '[MURDER]', 'Werewolf' : '[WEREWOLF]', 'Apocalypse' : '[APOCALYPSE]',
              'Haunted House' : '[HAUNTED HOUSE]', 'Witch' : '[WITCH]', 'Hell' : '[HELL]', 'Alien' : '[ALIEN]', 'Gore' : '[GORE]', 'Monster' : '[MONSTER]'}


author_tokens = {'Clive Barker' : '[CLIVE BARKER]', 'J. K. Rowling' : '[J.K. ROWLING]', 'Stephen King' : '[STEPHEN KING]', 'Théophile Gautier' : '[THEOPHILE GAUTIER]', 
               'James H. Hyslop' : '[JAMES H HYSLOP]', 'Lord Edward Bulwer-Lytton' : '[LORD EDWARD BULWER-LYTTON]', 'A. T. Quiller-Couch' : '[A. T. QUILLER-COUCH]', 
               'Mrs. Margaret Oliphant' : '[MRS. MARGARET OLIPHANT]', 'Ernest Theodor Amadeus Hoffmann' : '[ERNEST THEODOR AMADEUS HOFFMAN]', 'Erckmann-Chatrian' : '[ERCKMANN-CHATRAIN]', 
               'Fiona Macleod' : '[FIONA MACLEOD]', 'Amelia B. Edwards' : '[AMELIA B. EDWARDS]', 'H. B. Marryatt' : '[H. B. MARRYATT]', 'Thomas Hardy' : '[THOMAS HARDY]', 
               'Montague Rhodes James' : '[MONTAGUE RHODES JAMES]', 'Fitz-James O\'Brien' : '[FITZ-JAMES O\'BRIEN', 'James Stephen' : '[JAMES STEPHEN]', 'Alfred Lord Tennyson' : '[ALFRED LORD TENNYSON]',
               'Amelia Edwards' : '[AMELIA EDWARDS]', 'Edward Bulwer-Lytton' : '[EDWARD BULWER-LYTTON]', 'Erckmann Chatrian' : '[ERCKMANN CHATRIAN]', 'Latifa al-Zayya' : '[LATIFA AL-ZAYYA]',
               'M. R. James' : '[M. R. JAMES]', 'Paul Brandis' : '[PAUL BRANDIS]', 'Brain Evenson' : '[BRAIN EVENSON]', 'Elliott O\'Donnell' : '[ELLIOTT O\'DONNELL]', 
               'Joseph, Sheridan Le Fanu' : '[JOSEPH, SHERIDAN LE FANU]', 'Edgar Allan Poe' : '[EDGAR ALLEN POE]', 'Bram Stoker' : '[BRAM STOKER]', 'Algernon Blackwood' :'[ALGERNON BLACKWOOD]',
               'Miles Klee' : '[MILES KLEE]', 'Nnedi Okorador' : '[NNEDI OKORADOR]', 'Sofia Samatar' : '[SOFIA SAMATAR]', 'Franz Kafka' : '[FRANZ KAFKA]', 'Laird Barron' : '[LAIRD BARRON]',
               'Nathan Ballingrud' : '[NATHAN BALLINGRUD]', 'Nellie Bly' : '[NELLIE BLY]', 'William Hop Hodgson' : '[WILLIAM HOP HODGSON]', 'Ambrose Bierce' : '[AMBROSE BIERCE]',
               'Kelly Link' : '[KELLY LINK]', 'Arthur Machen' : '[ARTHUR MACHEN]', 'George Sylvester Viereck' : '[GEORGE SYLVESTER VIERECK]', 'Robert Chambers' : '[ROBERT CHAMBERS]',
               'John Meade Falkner' : '[JOHN MEADE FALKNER]', 'Ann Radcliffe' : '[ANN RADCLIFFE]', 'Howard Lovecraft' : '[HOWARD LOVECRAFT]', 'Louis Stevenson' : '[LOUIS STEVENSON]',
               'Edith Birkhead' : '[EDITH BIRKHEAD]', 'Jeff Vandermeer' : '[JEFF VANDERMEER]', 'Henry James' : '[HENRY JAMES]', 'John William Polidori' : '[JOHN WILLIAM POLIDORI]',
               'Bob Holland' : '[BOB HOLLAND]', 'Oliver Onions' : '[OLIVER ONIONS]'}
               
SPECIAL_TOKENS = list({**literary_tokens, **subgenre_tokens, **author_tokens}.values())
GPT2_MODEL_PATH = 'Models/gpt2_large_horror_generator/latest_model/'
GPT2_TOKENIZER_PATH = 'Models/gpt2_large_horror_generator/tokenizer-default/'

/content/drive/.shortcut-targets-by-id/19Z9fYLUW5aaRDu2a-rq0Ev2uiUl6J2ln/CSCI 470 Project/Epic


In [ ]:
gpt2_tokenizer = AutoTokenizer.from_pretrained(GPT2_TOKENIZER_PATH)
#gpt2_tokenizer = AutoTokenizer.from_pretrained("aadelucia/GPT2_medium_narrative_finetuned_large", additional_special_tokens=SPECIAL_TOKENS)
gpt2_model = AutoModelWithLMHead.from_pretrained(GPT2_MODEL_PATH, pad_token_id=gpt2_tokenizer.eos_token_id).to(device)
"""
zeroshot_tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")
zeroshot_model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli").to(device)
zeroshot_generator = pipeline("zero-shot-classification", device=0, model=zeroshot_model, tokenizer=zeroshot_tokenizer)
"""

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


'\nzeroshot_tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")\nzeroshot_model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli").to(device)\nzeroshot_generator = pipeline("zero-shot-classification", device=0, model=zeroshot_model, tokenizer=zeroshot_tokenizer)\n'

In [ ]:
bad_words_ids = get_bad_word_ids(gpt2_tokenizer)
for i, list in enumerate(bad_words_ids):
    if len(list) == 0:
        print(i, list)

In [ ]:
inputs = "[VAMPIRE][STEPHEN KING]The dog went for a walk on it's own to pick up a package of cigarettes from the gas station. He never returned."
generate_outputs(inputs, gpt2_model, gpt2_tokenizer, bad_words_ids[:-1], num_sequences=1, top_k=50, top_p=.7)

In [ ]:
? AutoModelWithLMHead.from_pretrained()

In [ ]:
gpt2_tokenizer.all_special_tokens

['<|startoftext|>', '<|endoftext|>', '[WP]', '[RESPONSE]']

In [ ]:
? AutoTokenizer.from_pretrained

In [ ]:
def find_sentence_locs(text):
    num_sentences = 0

    #find and store locations of quotations within text 
    dialogue_locations = []
    dialogue_pattern = r'"(?:(?:(?!(?<!\\)").)*)[.?!,]"'
    for match in re.finditer(dialogue_pattern, text):
        s = match.start()
        e = match.end()
        dialogue_locations.append((s, e))
        
    punc_locations = []
    for match in re.finditer("[!.?]", text):
        skip_punc = False
        punc_i = match.end() #get index of next punctuation

        #check if punctuation found was used in an honorific
        pat_obj = re.compile('(Mr)|(Mrs)|(Dr)|(Ms)|(Sr)|(Jr)|(Mt)', re.IGNORECASE)
        if pat_obj.search(text[punc_i-4: punc_i]):
            continue
        
        #if punctuation found is in between a set of quote, skip iteration       
        for s, e in dialogue_locations:
            if (s <= punc_i) and (punc_i <= e):
                skip_punc = True
                break
        if skip_punc == True:
            skip_punc = False
            continue 
        punc_locations.append(punc_i)

    return punc_locations

In [ ]:
def get_max_tokenizer(input, context, tokens, gpt2_tokenizer, entailment_tokenizer, sentiment_tokenizer):
    gpt2_tokens = gpt2_tokenizer.tokenize(tokens+context+input)
    entailment_tokens = entailment_tokenizer.tokenize(tokens+context+input)
    sentiment_tokens = sentiment_tokenizer.tokenize(tokens+context+input)

    max_tokenizer = gpt2_tokenizer
    max = len(gpt2_tokens)
    if len(entailment_tokens) > max:
        max_tokenizer = entailment_tokenizer
        max = len(entailment_tokens)
    if len(sentiment_tokens) > max:
        max_tokenizer = sentiment_tokenizer
        max = len(sentiment_tokens)

    return max_tokenizer

In [ ]:
def get_sequence(input, context, tokens, gpt2_tokenizer, entailment_tokenizer, sentiment_tokenizer):
    max_tokens = 510
    max_tokenizer = get_max_tokenizer(input, context, tokens, gpt2_tokenizer, entailment_tokenizer, sentiment_tokenizer)
    context_input_tokenized = max_tokenizer.tokenize(context+ " " + input)
    user_tokens_tokenized = gpt2_tokenizer.tokenize(tokens)
    if len(context_input_tokenized + user_tokens_tokenized) > max_tokens:
        index_for_slice = max_tokens - len(user_tokens_tokenized)
        context_input_tokenized = context_input_tokenized[-index_for_slice:]
        return gpt2_tokenizer.convert_tokens_to_string(user_tokens_tokenized) + max_tokenizer.convert_tokens_to_string(context_input_tokenized)
    else:
        return tokens + context + " " + input

In [ ]:
sentiment_tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
sentiment_generator = pipeline("sentiment-analysis",model=sentiment_model, tokenizer=sentiment_tokenizer, return_all_scores=True)

entailment_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
entailment_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
entailment_generator = pipeline("sentiment-analysis",model=entailment_model, tokenizer=entailment_tokenizer, return_all_scores=True)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import re

def match_sentiment(new_input, context, model, tokenizer, bad_words_ids, user_tokens=[]):
  #build a string of user defined tokens
  user_tokens = "".join(user_tokens)
  # Grab last couple sentences from context, if long enough. O/w grab context
  context_puncs = find_sentence_locs(context)
  context = context[context_puncs[-3]:] if len(context_puncs) >= 2 else context
  
  sequence = get_sequence(new_input, context, user_tokens, tokenizer, entailment_tokenizer, sentiment_tokenizer)
  
  # Pull sentiment and put star classification in sequence_sentiment
  sequence_max_score = 0
  for d in sentiment_generator(sequence.replace(user_tokens,''))[0]:
    if d['score'] > sequence_max_score:
        sequence_max_score = d['score']
        sentiment = d['label']
  sequence_sentiment = int(sentiment.split(' ')[0])

  accepted_samples = []
  while len(accepted_samples) < 1:
    # Generate outputs (out1 = without tokens, out2 = with tokens)
    out1, out2 = generate_outputs(sequence, model, tokenizer, bad_words_ids[:-1], num_sequences=3, top_k=25, top_p=.95, max_length=len(gpt2_tokenizer.tokenize(sequence))+35)

    accepted_samples = []
    #threshhold = 0.3
    for sample in out1:
      # Strip sequence from sample
      sample = sample.replace(sequence,'')
      # Strip tokens from sample
      sample = re.sub("[\[].*?[\]]",'',sample)
      #strip newlines  
      sample = sample.replace("\n", "")
      #strip backslash
      sample = sample.replace("newline>", "")
      #strip last bit
      punc_locations = find_sentence_locs(sample)
      sample = sample[:punc_locations[-1]] if len(punc_locations) > 0 else sample
      # Check sample sentiment, compare to sequence sentiment
      # Pull max score and save to sample_sentiment
      sample_max_score = 0
      sample_sentiment = ""
      for d in sentiment_generator(sample)[0]:
        if d['score'] > sample_max_score:
          sample_max_score = d['score']
          sample_sentiment = d['label']
      sample_sentiment = int(sentiment.split(' ')[0])
      if abs(sample_sentiment - sequence_sentiment) <= 1:
        # Accept sample
        entailment_sequence_tokens = entailment_tokenizer.tokenize(sequence.replace(user_tokens,'')+sample)
        entailment_sequence = entailment_tokenizer.convert_tokens_to_string(entailment_sequence_tokens[-510:])
        d = entailment_generator(entailment_sequence)[0][0] #check contradiction
        if d['score'] < 0.8:
            accepted_samples.append((sample,sample_max_score))

  if len(accepted_samples) >= 1:
    max = -1
    output = ""
    for sample, score in accepted_samples:
        if score > max:
            max = score
            output = sample

  output = new_input + " " + output
  output = ". ".join(output.split("."))
  output = "! ".join(output.split("!"))
  output = "? ".join(output.split("?"))
  output = " ".join(output.split())
  return output


In [ ]:
new_input = "The hippies were sad, but it was only Tuesday!"
context = "But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of human happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know how to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and pain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, except to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that has no annoying consequences, or one who avoids a pain that produces no resultant pleasure?"
new_input1 = "Far far away, behind the word mountains, far from the countries Vokalia and Consonantia, there live the blind texts. Separated they live in Bookmarksgrove right at the coast of the Semantics, a large language ocean. A small river named Duden flows by their place and supplies it with the necessary regelialia. It is a paradisematic country, in which roasted parts of sentences fly into your mouth. Even the all-powerful Pointing has no control about the blind texts it is an almost unorthographic life One day however a small line of blind text by the name of Lorem Ipsum decided to leave for the far World of Grammar. The Big Oxmox advised her not to do so, because there were thousands of bad Commas, wild Question Marks and devious Semikoli, but the Little Blind Text didn’t listen. She packed her seven versalia, put her initial into the belt and made herself on the way. When she reached the first hills of the Italic Mountains, she had a last view back on the skyline of her hometown Bookmarksgrove, the headline of Alphabet Village and the subline of her own road, the Line Lane. Pityful a rethoric question ran over her cheek, then "
new_input2 = """
The abyss is a steep wall, reaching left and right for as long as
anyone has ever travelled, up at least ten thousand levels from where I
sit, and (in myth) down to the depths of Sheol, where the air burns, and
the face of the abyss wall melts and runs perpetually downward, into a
molten infinity.  Standing on one of the switchbacked roads that men
have scratched into the wall, looking out away from the wall, there is
just darkness, fallers, and debris.  Down and out, an occasional red
glow from (one supposes) Sheol appears, but nothing else.  Sound from
the abyss is (in the absence of passers-by) only an occasional sighing,
as of distant winds, or very distant gales.
  Our scientists have determined that the light we see by, that wells in
the day and wanes in the night, comes from the rock of the wall, and the
air itself near to the wall.  Why it waxes and wanes, and what force can
cause the very air to give out with light, they cannot say.
  Men's houses sit on the wall like the round, dusty hills of the
spider's nests, clinging to the houses' walls.  Roads are hacked into
the rock, winding switchbacked between the houses, through towns, narrow
and crumbling in the wilderness.  Left to itself, the wall face is
irregular, bumpy, with long cracks and clefts, ledges leading to
nowhere, chimneys and hollows.
  The general tendency of each level is to slope inward (toward the
wall) at an angle comprising about one twentieth part of a circle from
the abyssward direction (the direction of the pull of gravity).  At the
border between one level and the next, there is an overhang, so that the
lowest parts of the upper level thrust outward into the void beyond the
upper parts of the lower level, and in fact beyond the lower parts of
the lower level, so that although each level slopes wallward, the
general tendency of the wall as a whole is to jut abyssward.  At least
this is what the legend says, and the dim looming greyness we see
looking up in the daytime, and the evidence of debris and fallers,
supports it.
  Passing from one level to the next lower is thus possible, although
neither easily nor without danger, by the means of long ropes and
pulleys or (if no assistance is to be had on the level below) by
swinging.  Travel in the opposite direction, from lower to higher
levels, is on the other hand all but impossible in the normal course of
things.  The rock of the border overhangs is tough and not subject to
cracks or fissures large enough to permit passage, yet has enough of a
tendency to flake and chip near the point that all efforts to establish
permanent bridges from this level to either of its neighbors have met
with (usually tragic) failure.
  The vertical height of the level varies, being generally between two
and five days travel from lower overhang to upper.  This does not take
into account the twists of our roads; if it were possible to move
directly against gravity, the distance would of course be much shorter.
  Because it is in general possible to cross the boundaries between
levels in only one direction, the general spread of humanity has been
downwards, towards (presumably) Sheol.	The population seems to be
distributed uniformly in the lateral direction (left and right across
the level), but to be spreading slowly downward between levels, as if
humanity had begun somewhere far, far above (and why else would the
levels be numbered, reckoned from up to down).	Strong tradition holds
that when one level becomes too crowded, the dangerous migration down to
the next becomes more common, until the lessening of population due to
bordercrossings (successful and unsuccessful) balances the birthrate.
As our people are never extremely fecund, this desertion rate need not
be high.
  There is some evidence to support popular tradition's picture of human
migration.  The villages near the top of the level tend to be the oldest
and best established, and the density of population there is higher.
   One of the favorite and longest-disputed topics among the
philosophers of this level is the fate of fallers.  Fallers are those
persons, or seeming persons, who pass by the level, more or less far out
in the abyss, on their way from somewhere above to somewhere below.
The popular wisdom states that the fate of fallers is simply to fall,
until they reach the levels of Sheol, and are melted to nothingness.
This is too simple for many of our wisest, however.  They are divided
into several schools.  One set of schools holds that, somewhere between
this level and the deadly nether reaches, there is something that brings
the fallers to a halt.	The schools disagree in the nature and placement
of these obstacles, the purposes of their creators (if any), whether or
not the obstacles are such that the fallers are destroyed upon meeting
them, and a host of other questions.
   One of the more notorious schools of faller theory, popular in our
great-grandfathers' day, held that ten thousand levels down, silken nets
bring the fallers to a gentle halt, and they are led off by servitors
(there was a schism early in the history of this theory on the question
of the gender, if any, of these) to await, in honored opulence, the day
when the normal migration of humanity reaches those regions.  This
school flourished in that time of optimism, but it tapered off as its
most staunch defenders journeyed to the lower edge of the level and
hurled themselves hopefully into space, equipped with greater or smaller
numbers of philosophic texts, missives, and holy runes intended to
ensure their friendly (not to say warm) reception in the Advanced
Regions.
"""
new_input3 = "this is a simple paragraph @#43 22243bw sfsfdsbd s@# that is meant to be nice and easy to type which is why th#!#1j23kblh 312;n312ere will be mommas no periods or any capital letters so i guess this means that it cannot really be considered a paragraph but just a series of run on sentences this should help you get faster at typing as im trying not to use too many difficult words in it although i think that i might start making it hard by including some more difficult letters I'm typing pretty quickly so forgive me for any mistakes i think that i will not just tell you a story about the time i went to the zoo and found a monkey and a fox playing together they were so cute and i think that they were not supposed to be in the same cage but they somehow were and i loved watching them horse around forgive the pun well i hope that it has been highly enjoyable typing this paragraph and i wish you the best of luck getting the best score that you possibly can "

print(match_sentiment(new_input3.replace(".", "").replace("?", "").replace("!", ""), new_input3.replace(".", "").replace("?", "").replace("!", ""), gpt2_model, gpt2_tokenizer, bad_words_ids, user_tokens=["[PROBLEM]", "[DANGER]", "[SURPRISE]", "[GHOST]"]))

this is a simple paragraph @#43 22243bw sfsfdsbd s@# that is meant to be nice and easy to type which is why th##1j23kblh 312;n312ere will be mommas no periods or any capital letters so i guess this means that it cannot really be considered a paragraph but just a series of run on sentences this should help you get faster at typing as im trying not to use too many difficult words in it although i think that i might start making it hard by including some more difficult letters I'm typing pretty quickly so forgive me for any mistakes i think that i will not just tell you a story about the time i went to the zoo and found a monkey and a fox playing together they were so cute and i think that they were not supposed to be in the same cage but they somehow were and i loved watching them horse around forgive the pun well i hope that it has been highly enjoyable typing this paragraph and i wish you the best of luck getting the best score that you possibly can this paragraph is meant for you to t

In [ ]:
text = "It was Tuesday the 23rd, the day after the fire at the Black Spot! It seemed like a day that would never end, and it seemed like the day that the whole world was going to end, but there was one thing that seemed certain, one thing which was going right on. The day was going good. The people of the world were going to make good on their promises. The day was good. They had gotten the book and they had gone to the library."
len(gpt2_tokenizer.tokenize(text))

97

In [ ]:
user_tokens = "[STEPHEN KING][VAMPIRE]"
text = "The dog went for a walk on it's own to pick up a package of cigarettes from the gas station. He never returned. It was a good dog, and he was glad to have it. It had a good place, and the mailman kept it clean. He got up to go to the door, paused, and looked back. The house was silent. He knew it was time to go, and it was all right. He had to. The dog was not a vampire, and there was no way he could get a vampire to follow him. And yet he could not"
text = text + text + text + text + text
text_tokenized = gpt2_tokenizer.tokenize(text)
user_tokens_tokenized = gpt2_tokenizer.tokenize(user_tokens)
#if len(text_tokenized + user_tokens_tokenized) > 512:
index_for_slice = 512 - len(user_tokens_tokenized)
len(text_tokenized[-index_for_slice:]) + len(user_tokens_tokenized)
new_context = text_tokenized[-index_for_slice:]
new_context

['Ġhad',
 'Ġto',
 '.',
 'ĠThe',
 'Ġdog',
 'Ġwas',
 'Ġnot',
 'Ġa',
 'Ġvampire',
 ',',
 'Ġand',
 'Ġthere',
 'Ġwas',
 'Ġno',
 'Ġway',
 'Ġhe',
 'Ġcould',
 'Ġget',
 'Ġa',
 'Ġvampire',
 'Ġto',
 'Ġfollow',
 'Ġhim',
 '.',
 'ĠAnd',
 'Ġyet',
 'Ġhe',
 'Ġcould',
 'Ġnot',
 'The',
 'Ġdog',
 'Ġwent',
 'Ġfor',
 'Ġa',
 'Ġwalk',
 'Ġon',
 'Ġit',
 "'s",
 'Ġown',
 'Ġto',
 'Ġpick',
 'Ġup',
 'Ġa',
 'Ġpackage',
 'Ġof',
 'Ġcigarettes',
 'Ġfrom',
 'Ġthe',
 'Ġgas',
 'Ġstation',
 '.',
 'ĠHe',
 'Ġnever',
 'Ġreturned',
 '.',
 'ĠIt',
 'Ġwas',
 'Ġa',
 'Ġgood',
 'Ġdog',
 ',',
 'Ġand',
 'Ġhe',
 'Ġwas',
 'Ġglad',
 'Ġto',
 'Ġhave',
 'Ġit',
 '.',
 'ĠIt',
 'Ġhad',
 'Ġa',
 'Ġgood',
 'Ġplace',
 ',',
 'Ġand',
 'Ġthe',
 'Ġmail',
 'man',
 'Ġkept',
 'Ġit',
 'Ġclean',
 '.',
 'ĠHe',
 'Ġgot',
 'Ġup',
 'Ġto',
 'Ġgo',
 'Ġto',
 'Ġthe',
 'Ġdoor',
 ',',
 'Ġpaused',
 ',',
 'Ġand',
 'Ġlooked',
 'Ġback',
 '.',
 'ĠThe',
 'Ġhouse',
 'Ġwas',
 'Ġsilent',
 '.',
 'ĠHe',
 'Ġknew',
 'Ġit',
 'Ġwas',
 'Ġtime',
 'Ġto',
 'Ġgo',
 ',',
 'Ġand',
 'Ġit',


In [ ]:
print(user_tokens_tokenized)

['[', 'STEP', 'H', 'EN', 'ĠKING', '][', 'V', 'AM', 'PI', 'RE', ']']


In [ ]:
gpt2_tokenizer.truncate_sequences(,)


In [ ]:
gpt2_tokenizer.convert_tokens_to_string(new_context)

" had to. The dog was not a vampire, and there was no way he could get a vampire to follow him. And yet he could notThe dog went for a walk on it's own to pick up a package of cigarettes from the gas station. He never returned. It was a good dog, and he was glad to have it. It had a good place, and the mailman kept it clean. He got up to go to the door, paused, and looked back. The house was silent. He knew it was time to go, and it was all right. He had to. The dog was not a vampire, and there was no way he could get a vampire to follow him. And yet he could notThe dog went for a walk on it's own to pick up a package of cigarettes from the gas station. He never returned. It was a good dog, and he was glad to have it. It had a good place, and the mailman kept it clean. He got up to go to the door, paused, and looked back. The house was silent. He knew it was time to go, and it was all right. He had to. The dog was not a vampire, and there was no way he could get a vampire to follow him

In [ ]:
encoded_text = gpt2_tokenizer.encode(text, add_special_tokens=True, max_length=512)
len(encoded_text)
gpt2_tokenizer.decode(encoded_text)

"The dog went for a walk on it's own to pick up a package of cigarettes from the gas station. He never returned. It was a good dog, and he was glad to have it. It had a good place, and the mailman kept it clean. He got up to go to the door, paused, and looked back. The house was silent. He knew it was time to go, and it was all right. He had to. The dog was not a vampire, and there was no way he could get a vampire to follow him. And yet he could notThe dog went for a walk on it's own to pick up a package of cigarettes from the gas station. He never returned. It was a good dog, and he was glad to have it. It had a good place, and the mailman kept it clean. He got up to go to the door, paused, and looked back. The house was silent. He knew it was time to go, and it was all right. He had to. The dog was not a vampire, and there was no way he could get a vampire to follow him. And yet he could notThe dog went for a walk on it's own to pick up a package of cigarettes from the gas station. 

In [ ]:
gpt2_tokenizer.encode("[DANGER][VAMPIRE]Hello there.")

[50270, 50282, 15496, 612, 13]

In [ ]:
thing = [("str1", 5), ("str2", 10)]
thing[-2]

('str1', 5)

In [ ]:
text = 'And there was nothing in his wallet, either.That was when he saw it? On the sidewalk by the bus Mr. adjkbda;fk dskjfla, "asdjkbad!"' 
puncs = find_num_sentences(text)

In [ ]:
text[-2:]

'!"'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
generator = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer, return_all_scores=True)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
 generator("The dog went for a walk on it's own to pick up a package of cigarettes from the gas station. The cat didn't.")

[[{'label': 'CONTRADICTION', 'score': 0.48397594690322876},
  {'label': 'NEUTRAL', 'score': 0.4758927524089813},
  {'label': 'ENTAILMENT', 'score': 0.04013129696249962}]]